In [1]:
import pandas as pd
from numpy import inf, NaN
from datetime import datetime
from os import listdir, path

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1 - Adding columns of calculated data

In [3]:
config = pd.read_csv('config.csv',index_col='var')
df = pd.read_csv(config.loc['raw_data'].path).head(2000)

In [30]:
#2 Checkpoint
df_formatted = df.copy()

# DATA TO BE USED BY TABLEAU

In [31]:
df_by_country = df_formatted.groupby(['Country/Region','Date']).sum().reset_index()

# columns over which the calculations will be performed
root_columns = ['Active','Confirmed','Deaths','Recovered']

# creating columns of daily percentage of increase in values:
for col in root_columns:
#    df_aux = df_by_country.groupby('Country/Region')

    col_daily_inc = col + "_daily_%inc_by_country"
#    df_by_country[country_daily_inc] = df_aux[col].pct_change().replace([inf, NaN], 0)*100
        
    col_new_cases = col + '_new_cases'
    col_new_cases_inc = col + '_new_cases_inc_rate'
    col_new_cases_inc_speed = col + '_new_cases_inc_rate_speed'
    
    df_by_country[col_new_cases] = (df_by_country[col] - 
                                    df_by_country[col].shift(periods=1)
                                   ).fillna(value=0)
    df_by_country[col_daily_inc] = df_by_country[col].pct_change().replace([inf, NaN], 0)*100
    
    # 1st derivative of column datas. It represents the rate of change in new cases:
    df_by_country[col_new_cases_inc] = (df_by_country[col_new_cases] - 
                                        df_by_country[col_new_cases].shift(periods=1)
                                       ).fillna(value=0)        
    
    # 2nd derivative of column datas. It represents the acceleration of the rate of change in new cases:
    df_by_country[col_new_cases_inc_speed] = (df_by_country[col_new_cases_inc] -
                                              df_by_country[col_new_cases_inc].shift(periods=1)
                                              ).fillna(value=0)

In [32]:
df_by_country.head(20)

,Country/Region,Date,Active,Confirmed,Deaths,Latitude,Longitude,Recovered,Active_new_cases,Active_daily_%inc_by_country,Active_new_cases_inc_rate,Active_new_cases_inc_rate_speed,Confirmed_new_cases,Confirmed_daily_%inc_by_country,Confirmed_new_cases_inc_rate,Confirmed_new_cases_inc_rate_speed,Deaths_new_cases,Deaths_daily_%inc_by_country,Deaths_new_cases_inc_rate,Deaths_new_cases_inc_rate_speed,Recovered_new_cases,Recovered_daily_%inc_by_country,Recovered_new_cases_inc_rate,Recovered_new_cases_inc_rate_speed
0,Australia,2020-01-23 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Australia,2020-01-25 00:00:00,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.000000,4.0,4.0,4.0,0.000000,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,2020-01-26 00:00:00,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,-4.0,-8.0,0.0,0.000000,-4.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Australia,2020-01-27 00:00:00,5.0,5.0,0.0,0.0,0.0,0.0,1.0,25.000000,1.0,5.0,1.0,25.000000,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Australia,2020-01-28 00:00:00,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.0,-2.0,0.0,0.000000,-1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Australia,2020-01-29 00:00:00,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Australia,2020-01-30 00:00:00,7.0,9.0,0.0,0.0,0.0,2.0,2.0,40.000000,2.0,2.0,4.0,80.000000,4.0,4.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0
7,Australia,2020-01-31 00:00:00,7.0,9.0,0.0,0.0,0.0,2.0,0.0,0.000000,-2.0,-4.0,0.0,0.000000,-4.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-4.0
8,Australia,2020-02-01 00:00:00,10.0,12.0,0.0,0.0,0.0,2.0,3.0,42.857143,3.0,5.0,3.0,33.333333,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9,Australia,2020-02-02 00:00:00,10.0,12.0,0.0,0.0,0.0,2.0,0.0,0.000000,-3.0,-6.0,0.0,0.000000,-3.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# DATA FOR PYTHON USE

In [5]:
# drop columns that won't be used in the calculations:
column_labels = ['Last Update','Latitude','Longitude','Year','Month','Week','Day','Mortality rate in %']
df_formatted.drop(axis=1,inplace=True,labels=column_labels)

df_by_country = df_formatted.groupby(['Country/Region','Date']).sum().reset_index()

# Formatting dataframe for further calculations:
root_columns = ['Active','Confirmed','Deaths','Recovered']       # columns over which the calculations will be performed
MA = [3,7,15]                                                    # Moving Average intervals for computation

# creating columns of daily percentage of increase in values:
for col in root_columns:
    df_aux = df_by_country.groupby('Country/Region')

    country_daily_inc = col + "_daily_%inc_by_country"
   
    df_by_country[country_daily_inc] = df_aux[col].pct_change().fillna(value=0)*100
    for interval in MA:
        interval_col_name = col + "_" + str(interval) + "day_MA"
        df_by_country[interval_col_name] = df_aux.rolling(interval)[col].mean().fillna(method='bfill').reset_index(drop=True)    
        
        if interval == min(MA):
            column_name_first = col + '_1st_derivative'
            column_name_second = col + '_2nd_derivative'

            df_by_country[column_name_first] = (df_by_country[interval_col_name] - df_by_country[interval_col_name].shift(periods=1)).fillna(method='bfill')
            df_by_country[column_name_second] = (df_by_country[column_name_first] - df_by_country[column_name_first].shift(periods=1)).fillna(method='bfill')        

In [6]:
df_by_country.sort_values(by=['Country/Region','Date']).head(5)

,Country/Region,Date,Active,Confirmed,Deaths,Recovered,Active_daily_%inc_by_country,Active_3day_MA,Active_1st_derivative,Active_2nd_derivative,Active_7day_MA,Active_15day_MA,Confirmed_daily_%inc_by_country,Confirmed_3day_MA,Confirmed_1st_derivative,Confirmed_2nd_derivative,Confirmed_7day_MA,Confirmed_15day_MA,Deaths_daily_%inc_by_country,Deaths_3day_MA,Deaths_1st_derivative,Deaths_2nd_derivative,Deaths_7day_MA,Deaths_15day_MA,Recovered_daily_%inc_by_country,Recovered_3day_MA,Recovered_1st_derivative,Recovered_2nd_derivative,Recovered_7day_MA,Recovered_15day_MA
0,Azerbaijan,2020-02-28 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,2020-02-24 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,2020-02-25 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,2020-02-26 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,2020-02-27 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_by_country.to_json('country_report.json',orient='records')

In [198]:
# Calculating the number of days since the 1st case:
df_by_country['Days_since_1st_case'] = pd.to_datetime(df_by_country.groupby('Country/Region')['Date'].first())
df_by_country['Days_since_1st_case'] = (df_by_country['Date'] - df_by_country['Days_since_1st_case']).dt.days

TypeError: cannot subtract DatetimeArray from ndarray

In [10]:
# Formatting dataframe for further calculations:
root_columns = ['Confirmed','Deaths','Recovered','Active']       # columns over which the calculations will be performed
sort_columns = ['Country/Region','Province/State','Date']        # order for sorting columns
MA = [3,7,15]                                                    # Moving Average intervals for computation

# pre-formatting frame:
group = ['Country/Region','Province/State','Month','Week','Day']
df_formatted = df_formatted.groupby(group).max().sort_values(by=sort_columns,axis=0)

# creating columns of daily percentage of increase in values:
for col in root_columns:
    daily_inc = col + "_daily_%inc"
    df_formatted[daily_inc] = df_formatted[col].pct_change().fillna(value=0)*100

# creating columns of moving average in root columns values:
group = ['Country/Region','Province/State']
for col in root_columns:
    for interval in MA:
        interval_col_name = col + "_" + str(interval) + "d_MA"
        df_formatted[interval_col_name] = df_formatted.groupby(group).rolling(interval)[col].mean().droplevel(level=[0,1]).fillna(method='bfill')

# Calculating the number of days since the 1st case:
df_formatted['Days_since_1st_case'] = pd.to_datetime(df_formatted.groupby(['Country/Region','Province/State'])['Date'].first())
df_formatted['Days_since_1st_case'] = (df_formatted['Date'] - df_formatted['Days_since_1st_case']).dt.days


# Calculating 1st and 2nd derivatives for root_columns:

# Since the derivative of a function is subject to noise the calculation will be performed over 
# the mean data (taking the minimum interval) in order to minimize the noise influence

interval = str(min(MA)) # The value of the minimum interval
for col in root_columns:
    column_name_first = col + '_1st_derivative'
    column_name_second = col + '_2nd_derivative'
    column_data = col + '_' + interval + "d_MA"
    df_formatted[column_name_first] = (df_formatted[column_data] - df_formatted[column_data].shift(periods=1)).fillna(method='bfill')
    df_formatted[column_name_second] = (df_formatted[column_name_first] - df_formatted[column_name_first].shift(periods=1)).fillna(method='bfill')

In [18]:
df_formatted.to_excel('formatted_data.xlsx')

# 4 - Ordering & Sorting the DataFrame columns

In [31]:
columns = ['Date', 'Country/Region', 'Province/State', 'Latitude', 'Longitude',
           'Confirmed', 'Deaths', 'Recovered', 'Active','Year', 'Month', 'Week',
           'Day','Last Update']

df_formatted = df_formatted[columns]

# 5 - Writing dataframe into external file 

In [9]:
df_formatted.size

2091565